In [1]:
import numpy as np 
import pandas as pd

In [2]:
train = pd.read_csv('../input/weather-dataset-rattle-package/weatherAUS.csv') 

In [3]:
train['year'] = train['Date'].str[:4]  #This will get the first four letters from train..Nothing nothing
train['month'] = train['Date'].str[5:7]
train['day'] = train['Date'].str[8:10]

In [4]:
train = train.drop('Date',axis=1)

In [5]:
train['windDirSame'] = 0 
train.loc[train['WindDir9am']==train['WindDir3pm'],'windDirSame'] = 1

In [6]:
#Wind Speed, Humidity, Pressure, Cloud, Temperature 
train['windDiff'] = train['WindSpeed9am'] - train['WindSpeed3pm']
train['humidityDiff'] = train['Humidity9am'] - train['Humidity3pm']
train['pressureDiff'] = train['Pressure9am'] - train['Pressure3pm']
train['CloudDiff'] = train['Cloud9am'] - train['Cloud3pm']
train['TempDiff'] = train['Temp9am'] - train['Temp3pm']

In [7]:
train.head()

,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,...,RainTomorrow,year,month,day,windDirSame,windDiff,humidityDiff,pressureDiff,CloudDiff,TempDiff
0,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,WNW,...,No,2008,12,01,0,-4.0,49.0,0.6,NaN,-4.9
1,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,WSW,...,No,2008,12,02,0,-18.0,19.0,2.8,NaN,-7.1
2,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,WSW,...,No,2008,12,03,0,-7.0,8.0,-1.1,NaN,-2.2
3,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,E,...,No,2008,12,04,0,2.0,29.0,4.8,NaN,-8.4
4,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,NW,...,No,2008,12,05,0,-13.0,49.0,4.8,-1.0,-11.9


In [8]:
train['RainTomorrow'] = train['RainTomorrow'].astype('category').cat.codes
train['RainToday'] = train['RainToday'].astype('category').cat.codes
#train = pd.get_dummies(train)
train.shape

(142193, 32)

In [9]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 142193 entries, 0 to 142192
Data columns (total 32 columns):
Location         142193 non-null object
MinTemp          141556 non-null float64
MaxTemp          141871 non-null float64
Rainfall         140787 non-null float64
Evaporation      81350 non-null float64
Sunshine         74377 non-null float64
WindGustDir      132863 non-null object
WindGustSpeed    132923 non-null float64
WindDir9am       132180 non-null object
WindDir3pm       138415 non-null object
WindSpeed9am     140845 non-null float64
WindSpeed3pm     139563 non-null float64
Humidity9am      140419 non-null float64
Humidity3pm      138583 non-null float64
Pressure9am      128179 non-null float64
Pressure3pm      128212 non-null float64
Cloud9am         88536 non-null float64
Cloud3pm         85099 non-null float64
Temp9am          141289 non-null float64
Temp3pm          139467 non-null float64
RainToday        142193 non-null int8
RISK_MM          142193 non-null float6

In [10]:
train = pd.get_dummies(train)

In [11]:
meanRain = train.loc[train['RISK_MM'] != 0]['RISK_MM'].mean()
medianRain = train.loc[train['RISK_MM'] != 0]['RISK_MM'].median()

print("Mean Rain",meanRain)
print("Median Rain",medianRain)

Mean Rain 6.566875342359574
Median Rain 2.2


In [12]:
#Let's divide the rain into 4 categories

#If Rain is above mean, then it will be high
#If the rain is between median and mean, it will be medium
#If the rain is below median but above 0 then light

#Make a new column named rain Category
train['rainCategory'] = 0
#If rain is light, assign it 1
train.loc[ (train['RISK_MM'] != 0) & (train['RISK_MM'] < medianRain)  ,'rainCategory'] = 1
#If rain is medium, assign it 2
train.loc[ (train['RISK_MM'] > medianRain) & (train['RISK_MM'] < meanRain)  ,'rainCategory'] = 2
#If rain is high, assign it 3
train.loc[train['RISK_MM'] > meanRain  ,'rainCategory'] = 3

In [13]:
X_train = train.loc[(train['year_2016'] != 1) & (train['year_2017'] !=  1)]

test = train.loc[(train['year_2016'] == 1) | (train['year_2017'] == 1)]

In [14]:
y = X_train['rainCategory']
X_train = X_train.drop(['RainTomorrow','RISK_MM','rainCategory'],axis=1)

y_test = test['rainCategory'] 
test = test.drop(['RainTomorrow','RISK_MM','rainCategory'],axis=1)

In [15]:
import xgboost as xgb
#train_set = lgb.Dataset(X_train, label=y)
#valid_set = lgb.Dataset(test, label=y_test)

In [16]:
dtrain = xgb.DMatrix(X_train, label=y)
dval= xgb.DMatrix(test, label=y_test)

params = {
             "learning_rate":0.01,
             "eval_metric":'merror',
             "max_depth":8,
             "verbose":1,
             "nthread":4,
             "objective":"multi:softmax",
            'num_class':4
}

model = xgb.train(params, dtrain, evals=[(dtrain, "train"),(dval, 'val')], num_boost_round = 1200)

/opt/conda/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/opt/conda/lib/python3.6/site-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


[0]	train-merror:0.279713	val-merror:0.306191
[1]	train-merror:0.279171	val-merror:0.305613
[2]	train-merror:0.279464	val-merror:0.30519
[3]	train-merror:0.279885	val-merror:0.305382
[4]	train-merror:0.278758	val-merror:0.305305
[5]	train-merror:0.27881	val-merror:0.305074
[6]	train-merror:0.278801	val-merror:0.305613
[7]	train-merror:0.278801	val-merror:0.305305
[8]	train-merror:0.278956	val-merror:0.305036
[9]	train-merror:0.278337	val-merror:0.305228
[10]	train-merror:0.278242	val-merror:0.304343
[11]	train-merror:0.27801	val-merror:0.304689
[12]	train-merror:0.278121	val-merror:0.304189
[13]	train-merror:0.277511	val-merror:0.303534
[14]	train-merror:0.277072	val-merror:0.303342
[15]	train-merror:0.276624	val-merror:0.303111
[16]	train-merror:0.276211	val-merror:0.302764
[17]	train-merror:0.275988	val-merror:0.302225
[18]	train-merror:0.27542	val-merror:0.302187
[19]	train-merror:0.27511	val-merror:0.30261
[20]	train-merror:0.274749	val-merror:0.302341
[21]	train-merror:0.274396	va

In [17]:
dtest = xgb.DMatrix(test, label=y)
pred = model.predict(dtest)
pred

array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)

In [18]:
from sklearn.metrics import accuracy_score
pred = np.round(pred)
print(accuracy_score(y_test, pred))

0.7085162085162086
